In [1]:
import pandas as pd
import numpy as np


In [2]:
np.random.seed(42)


In [3]:
columns = [
    "avg_steps_7d",
    "avg_sleep_7d",
    "avg_water_7d",
    "missed_days_7d",
    "steps_variance",
    "sleep_variance",
    "prev_week_score",
    "next_week_consistency_score"
]

df = pd.DataFrame(columns=columns)
df


,avg_steps_7d,avg_sleep_7d,avg_water_7d,missed_days_7d,steps_variance,sleep_variance,prev_week_score,next_week_consistency_score


In [4]:
rows = 1000


In [5]:
df["avg_steps_7d"] = np.random.randint(2000, 15000, rows)


In [6]:
df["avg_sleep_7d"] = np.round(np.random.uniform(4.5, 9.0, rows), 1)


In [7]:
df["avg_water_7d"] = np.round(np.random.uniform(1.0, 4.5, rows), 1)


In [8]:
df["missed_days_7d"] = np.random.randint(0, 4, rows)


In [9]:
df["steps_variance"] = np.round(np.random.uniform(500, 4000, rows), 1)


In [10]:
# fill remaining feature
df["sleep_variance"] = np.round(np.random.uniform(0.2, 2.5, rows), 2)
df["prev_week_score"] = np.random.randint(40, 95, rows)

# create target: next week consistency score
df["next_week_consistency_score"] = (
    0.35 * df["prev_week_score"] +
    0.25 * (df["avg_sleep_7d"] / 9.0) * 100 +
    0.20 * (df["avg_steps_7d"] / 15000) * 100 +
    0.20 * (1 - df["missed_days_7d"] / 3) * 100
).clip(0, 100)

df.head()


,avg_steps_7d,avg_sleep_7d,avg_water_7d,missed_days_7d,steps_variance,sleep_variance,prev_week_score,next_week_consistency_score
0,9270,6.1,2.4,2,3615.2,1.99,90,67.471111
1,2860,6.1,2.3,2,795.1,1.75,89,58.574444
2,7390,4.6,2.2,0,3325.2,1.87,93,75.181111
3,7191,7.0,3.2,3,3514.6,1.33,57,48.982444
4,13964,6.9,2.5,0,3360.4,1.66,85,87.535333


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# split data
X = df.drop(columns=["next_week_consistency_score"])
y = df["next_week_consistency_score"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# train model
model = LinearRegression()
model.fit(X_train, y_train)

# evaluate
predictions = model.predict(X_test)
mae = mean_absolute_error(y_test, predictions)

mae


5.659472890329198e-14

In [12]:
sample_user = pd.DataFrame([{
    "avg_steps_7d": 9000,
    "avg_sleep_7d": 7.5,
    "avg_water_7d": 3.0,
    "missed_days_7d": 1,
    "steps_variance": 1200,
    "sleep_variance": 0.6,
    "prev_week_score": 72
}])

model.predict(sample_user)


array([71.36666667])

In [13]:
import joblib

joblib.dump(model, "habit_consistency_model.pkl")
print("Model saved successfully")


Model saved successfully


In [14]:
def generate_explanation(input_data, prediction):
    reasons = []

    if input_data["avg_sleep_7d"] < 6:
        reasons.append("low sleep consistency")
    elif input_data["avg_sleep_7d"] >= 7:
        reasons.append("good sleep habits")

    if input_data["missed_days_7d"] >= 2:
        reasons.append("frequent missed days")
    else:
        reasons.append("good habit adherence")

    if input_data["avg_steps_7d"] >= 8000:
        reasons.append("active lifestyle")

    explanation = (
        f"Your predicted consistency score is {prediction:.1f}. "
        f"This is mainly influenced by " + ", ".join(reasons) + "."
    )

    return explanation


In [15]:
user_input = {
    "avg_steps_7d": 9000,
    "avg_sleep_7d": 7.5,
    "avg_water_7d": 3.0,
    "missed_days_7d": 1,
    "steps_variance": 1200,
    "sleep_variance": 0.6,
    "prev_week_score": 72
}

input_df = pd.DataFrame([user_input])
prediction = model.predict(input_df)[0]

generate_explanation(user_input, prediction)


'Your predicted consistency score is 71.4. This is mainly influenced by good sleep habits, good habit adherence, active lifestyle.'

In [16]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.8 MB/s eta 0:00:00


In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import joblib

np.random.seed(42)

rows = 1000
df = pd.DataFrame({
    "avg_steps_7d": np.random.randint(2000, 15000, rows),
    "avg_sleep_7d": np.random.uniform(4.5, 9.0, rows),
    "avg_water_7d": np.random.uniform(1.0, 4.5, rows),
    "missed_days_7d": np.random.randint(0, 4, rows),
    "steps_variance": np.random.uniform(500, 4000, rows),
    "sleep_variance": np.random.uniform(0.2, 2.5, rows),
    "prev_week_score": np.random.randint(40, 95, rows),
})

df["next_week_consistency_score"] = (
    0.35 * df["prev_week_score"] +
    0.25 * (df["avg_sleep_7d"] / 9.0) * 100 +
    0.20 * (df["avg_steps_7d"] / 15000) * 100 +
    0.20 * (1 - df["missed_days_7d"] / 3) * 100
).clip(0, 100)

X = df.drop(columns=["next_week_consistency_score"])
y = df["next_week_consistency_score"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = LinearRegression()
model.fit(X_train, y_train)

print("MAE:", round(mean_absolute_error(y_test, model.predict(X_test)), 2))

joblib.dump(model, "habit_model.pkl")


MAE: 0.0


['habit_model.pkl']

In [18]:
import gradio as gr
import pandas as pd
import joblib

model = joblib.load("habit_model.pkl")

def predict(
    avg_steps, avg_sleep, avg_water,
    missed_days, steps_var, sleep_var, prev_score
):
    df = pd.DataFrame([{
        "avg_steps_7d": avg_steps,
        "avg_sleep_7d": avg_sleep,
        "avg_water_7d": avg_water,
        "missed_days_7d": missed_days,
        "steps_variance": steps_var,
        "sleep_variance": sleep_var,
        "prev_week_score": prev_score
    }])

    prediction = model.predict(df)[0]

    explanation = (
        f"Your predicted habit consistency score is "
        f"{prediction:.1f}/100.\n\n"
        f"This prediction is influenced by sleep regularity, "
        f"activity level, and habit adherence."
    )

    return prediction, explanation


custom_css = """
/* --- PAGE BACKGROUND (already approved) --- */
body {
    min-height: 100vh;
    background: linear-gradient(
        180deg,
        #DEDEEA 0%,
        #DEDEEA 60%,
        #E6E5F2 100%
    );
    background-attachment: fixed;
}

.gradio-container {
    background: transparent !important;
}

/* --- BIG BOXED HEADING --- */
.hero-box {
    background: rgba(222, 222, 234, 0.85); /* #DEDEEA */
    border: 1.5px solid #BCB9D8;
    border-radius: 24px;
    padding: 36px 40px;
    margin-bottom: 28px;
    box-shadow:
        0 20px 40px rgba(97, 103, 139, 0.18);
    transition: transform 0.35s ease, box-shadow 0.35s ease;
}

/* Zoom OUT effect on hover / touch */
.hero-box:hover {
    transform: scale(0.96);
    box-shadow:
        0 10px 24px rgba(97, 103, 139, 0.15);
}

/* Heading text */
.hero-title {
    font-size: 2.4rem;
    font-weight: 700;
    color: #565471;
    margin-bottom: 8px;
}

.hero-subtitle {
    font-size: 1.05rem;
    color: #61678B;
    opacity: 0.9;
}
"""




with gr.Blocks(css=custom_css) as demo:

    gr.Markdown(
        """
        # 🧠 ML-Based Health Habit Analyzer
        Predict your **next-week habit consistency** using a trained **machine learning model**.
        """
    )

    with gr.Row():
        with gr.Column(elem_classes="glass-card"):
            gr.Markdown("### 🔢 Input Your Weekly Habits")

            avg_steps = gr.Slider(2000, 15000, 9000, label="Average Daily Steps")
            avg_sleep = gr.Slider(4, 9, 7.5, label="Average Sleep (hours)")
            avg_water = gr.Slider(1, 4.5, 3.0, label="Water Intake (liters)")
            missed_days = gr.Slider(0, 3, 1, step=1, label="Missed Habit Days")
            steps_var = gr.Slider(500, 4000, 1200, label="Steps Variance")
            sleep_var = gr.Slider(0.2, 2.5, 0.6, label="Sleep Variance")
            prev_score = gr.Slider(40, 95, 72, label="Previous Week Score")

            predict_btn = gr.Button("✨ Analyze & Predict")

        with gr.Column(elem_classes="glass-card"):
            gr.Markdown("### 📊 Prediction Output")

            output_score = gr.Number(label="Predicted Consistency Score")
            output_text = gr.Textbox(
                label="AI Explanation",
                lines=5
            )

    predict_btn.click(
        predict,
        inputs=[
            avg_steps, avg_sleep, avg_water,
            missed_days, steps_var, sleep_var, prev_score
        ],
        outputs=[output_score, output_text]
    )

demo.launch()


/tmp/ipython-input-1852482193.py:87: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=custom_css) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://395f8b0a921009073d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
